In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# TL;DR

| Model                                       | R-squared (train)  | R-squared (val)    | R-squared (test)   | R-absolute (train) | R-absolute (val)   | R-absolute (test)  |
|---------------------------------------------|--------------------|--------------------|--------------------|--------------------|--------------------|--------------------|
| Linear regression                           | 0.5169866          |                    | 0.4549708          | 0.3383185          |                    | 0.3255524          |
| Lasso                                       | 0.5166027          |                    | 0.4613136          | 0.3371221          |                    | 0.3272954          |
| LARS                                        | 0.5160043          |                    | 0.4642462          | 0.3359979          |                    | 0.3276234          |
| Polynomial regression                       | 0.6010396          |                    | 0.5301241          | 0.3994169          |                    | 0.3740960          |
| Linear regression with RFE (6 features)     | 0.5160676          |                    | 0.4575793          | 0.3379842          |                    | 0.3255046          |
| Polynomial regression with RFE (5 features) | 0.4552918          |                    | 0.4691225          | 0.2864168          |                    | 0.3326174          |
| CrossNet                                    | 0.6884830          | 0.2226251          | 0.5017266          | 0.4879999          | 0.1206011          | 0.3968694          |
| MLP                                         | 0.9980898          | 0.1031491          | 0.2870642          | 0.9613156          | 0.1044721          | 0.2413247          |
| XGBoost                                     | 0.6163958          | 0.3052738          | 0.4813085          | 0.4093728          | 0.1779834          | 0.3262125          |
| EBM                                         | 0.6947590          | 0.4140548          | 0.4582469          | 0.4723313          | 0.2118888          | 0.3200194          |
| NAM                                         | 0.4631733          | 0.4038129          | 0.4546912          | 0.2804932          | 0.1968498          | 0.2907707          |
| NODE GAM                                    | 0.6060284          | 0.6050312          | 0.4252885          | 0.4231943          | 0.3740149          | 0.3055096          |
| NODE GA2M                                   | 0.4647184          | 0.4057951          | 0.3696757          | 0.3209596          | 0.3050901          | 0.2833377          |
| HONAM                                       | 0.9932572          | -1.1341181         | -0.7910015         | 0.9315096          | -0.3565330         | -0.2370180         |


# TODO in report
- add y transformation in methodology
- add info about batch size

- https://hastie.su.domains/Papers/LARS/LeastAngle_2002.pdf
- http://rstudio-pubs-static.s3.amazonaws.com/358961_ddc67a63195d43ed981b81f5bed43884.html
- https://strimmerlab.github.io/software/care/download/diabetes.pdf
- http://garthtarr.github.io/mplot/articles/diabetes.html
- https://github.com/federico-trotta/predicting_diabetes_with_ML
- LARS


In [2]:
import numpy as np
import pandas as pd
import torch
from utils import evaluate
from outliers import outliers_box

## Dataset

[Link to dataset](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html)

In [52]:
df = pd.read_csv('./data/diabetes_data.csv')
df.head()

,age,sex,bmi,map,tc,ldl,hdl,tch,ltg,glu,y
0,59,2,32.1,101.0,157,93.2,38.0,4.0,4.8598,87,151
1,48,1,21.6,87.0,183,103.2,70.0,3.0,3.8918,69,75
2,72,2,30.5,93.0,156,93.6,41.0,4.0,4.6728,85,141
3,24,1,25.3,84.0,198,131.4,40.0,5.0,4.8903,89,206
4,50,1,23.0,101.0,192,125.4,52.0,4.0,4.2905,80,135


In [ ]:
df.describe()

In [ ]:
df.applymap(lambda x: x == 0).sum()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 15))

mask = np.triu(np.ones_like(df.corr()))
ax = plt.axes()
sns.heatmap(df.corr(), annot=True, fmt='.2f', mask=mask, cmap="RdYlGn", vmin=-1, vmax=1, )
ax.set_title('Correlation matrix for raw diabetes progression data')
plt.show()

In [ ]:
import plotly.express as px
fig = px.scatter(df, x="tc", y="ldl", color='y', color_continuous_scale='rdylgn', labels={'tc': 'total serum cholesterol', 'ldl': 'low-density lipoprotein', 'y': 'disease progression 1 year after baseline'},
                 title='Correlation between low-density lipoprotein (ldl) and total serum cholesterol (tc)', height=1200, width=2400, trendline='ols')
fig.show()

In [ ]:
import plotly.express as px
fig = px.scatter(df, x="tch", y="hdl", color='y', color_continuous_scale='rdylgn', labels={'tch': 'total cholesterol', 'hdl': 'high-density lipoprotein', 'y': 'disease progression 1 year after baseline'},
                 title='Correlation between high-density lipoprotein (hdl) and total cholesterol (tch)', height=1200, width=2400, trendline='ols', trendline_options={'log_x': True, 'log_y': True})
fig.show()

In [ ]:
import plotly.figure_factory as ff
fig = ff.create_scatterplotmatrix(df, diag='histogram', size=3, title='Pairwise correltation in raw diabetes progression data', height=1200, width=2400)
for fig_data in fig.data:
    fig_data['marker']['color'] = '#1f77b4'
fig.show()

### Quantile transformation comparison

In [ ]:
from plotly.subplots import make_subplots

from sklearn.preprocessing import QuantileTransformer, StandardScaler
import plotly.figure_factory as ff

transformer = QuantileTransformer(output_distribution='normal', n_quantiles=df.shape[0])
scaler = StandardScaler()
var = 'y'
trans = transformer.fit_transform(df[[var]])
scaled = scaler.fit_transform(df[[var]])
trans_scaled = scaler.fit_transform(trans)

fig = ff.create_distplot([scaled.ravel(), trans_scaled.ravel()], ['Without Quantile Transformation', 'With Quantile Transformation'], show_hist=False, bin_size=0.01, show_curve=True, show_rug=False)
fig.update_layout(title_text=f'{var}')
fig.update_yaxes(range=[0, 1])
fig.update_xaxes(range=[-5, 5])
fig.show()

# Data pre-processing

In [72]:
from load_data import load_diabetes_data
columns, (x_raw_train, y_raw_train), (x_raw_val, y_raw_val), (x_raw_test, y_raw_test) = load_diabetes_data()
print('COLUMNS: ', columns)
print('training set: ', x_raw_train.shape, y_raw_train.shape)
print('validation set: ', x_raw_val.shape, y_raw_val.shape)
print('test set: ', x_raw_test.shape, y_raw_test.shape)

COLUMNS:  ['age', 'sex', 'bmi', 'map', 'tc', 'ldl', 'hdl', 'tch', 'ltg', 'glu']
training set:  (266, 10) (266, 1)
validation set:  (88, 10) (88, 1)
test set:  (88, 10) (88, 1)


In [73]:
task = 'regression'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = int(np.ceil(x_raw_train.shape[0] / 100))
print(f'task={task}: device is {device} and batch size is {batch_size}')

task=regression: device is cpu and batch size is 3


In [74]:
from load_data import load_diabetes_data
from preprocessor import Preprocessor

preprocessor = Preprocessor(task=task, n_quantiles=x_raw_train.shape[0])
x_train, y_train = preprocessor.fit_transform(x_raw_train, y_raw_train)
x_val, y_val = preprocessor.transform(x_raw_val, y_raw_val)
x_test, y_test = preprocessor.transform(x_raw_test, y_raw_test)

# Baselining

## Statsmodel

In [8]:
labels = {
  "age": "age",
  "sex": "sex",
  "bmi": "body mass index",
  "map": "average blood pressure",
  "tc": "total serum cholesterol",
  "ldl": "low-density lipoprotein",
  "hdl": "high-density lipoprotein",
  "tch": "total cholesterol",
  "ltg": "lamotrigine",
  "glu": "glucose",
  "y": "disease progression one year after baseline"
}

In [119]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def get_lr_local_interpretation(index, y_hat, feature_weights):
    x = x_test[index]
    feature_influence = x * feature_weights
    x_true = preprocessor.inverse_transform_x([x])[0]
    y_true = preprocessor.inverse_transform_y([y_test[index]])[0][0]
    y_pred = preprocessor.inverse_transform_y([[y_hat]])[0][0]
    y_mean = preprocessor.inverse_transform_y([[0]])[0][0]

    df = pd.DataFrame(np.stack([x, x_true, feature_weights, feature_influence]), columns=columns).T.reset_index()
    df.columns = ['variable', 'scaled', 'original', 'feature_weights', 'feature_contribution']
    df['variable'] = df['variable'].apply(lambda x: labels[x])

    return df, int(y_true), y_pred, y_mean

def plot_local_interpretation(df, title):
    df['variable_val'] = df.apply(lambda r: f"{r['variable']}<br>({int(r['original']) if r['original'].is_integer() else round(r['original'], 2)})", axis=1)

    fig = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=("Feature contribution","Values of features and output relative to the mean"))

    fig.add_trace(go.Bar(x=df['feature_contribution'], y=df['variable'], orientation='h', text=df['variable'], showlegend=False, marker={'color':'#1f77b4'}), row=1, col=1)
    fig.add_trace(go.Bar(x=df['scaled'], y=df['variable'], orientation='h', text=df['variable_val'], showlegend=False, marker={'color':['#1f77b4' for i in range(10)] + ['#bcbd22', '#2ca02c']}, textposition='auto'),row=1, col=2)
    if 'feature_weights' in df.columns:
        fig.add_trace(go.Bar(x=df['feature_weights'], y=df['variable'], orientation='h', showlegend=False, marker={'color':'#17becf'}),row=1, col=2)

    fig.update_layout(title_text=title, height=800)
    fig.update_yaxes(visible=False, showticklabels=False)
    fig['layout']['xaxis1'].update(range=[-1,1], showgrid=False)
    fig['layout']['xaxis2'].update(range=[-5,5], showgrid=False)
    fig['layout'].update(barmode='relative')
    
    return fig

## Linear regression

In [120]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train, y_train)
lr_fit = lr.predict(x_train)
lr_pred = lr.predict(x_test)

print("TRAIN: ", evaluate(y_train, lr_fit, task))
print("TEST: ", evaluate(y_test, lr_pred, task))
lr_weights = pd.DataFrame([(col, coefficient) for col, coefficient in zip(columns, lr.coef_[-1])], columns=['variable', 'coefficient'])
lr_weights['variable'] = lr_weights['variable'].apply(lambda x: labels[x])
lr_weights.sort_values(by='coefficient', key=abs, ascending=False)

TRAIN:  (0.4835605991369081, 0.31706055160130386)
TEST:  (0.4153718035046221, 0.27259523421109166)


,variable,coefficient
2,body mass index,0.323797
6,high-density lipoprotein,-0.279573
3,average blood pressure,0.248764
8,lamotrigine,0.195404
1,sex,-0.155652
4,total serum cholesterol,0.138189
5,low-density lipoprotein,-0.128874
7,total cholesterol,-0.056285
9,glucose,0.019193
0,age,-0.007005


In [121]:
import plotly.express as px
fig = px.bar(lr_weights, y='variable', x='coefficient', orientation='h', range_x=(-0.5, 0.5), text='variable', title='Global interpretation of Linear Regression', height=800, width=800, labels={'coefficient': 'feature importance'})
fig.update_yaxes(visible=False, showticklabels=False)
fig.update_xaxes(showgrid=False)
fig.show()

In [122]:
true_negative = 74
false_negative = 46
true_positive = 76
false_positive = 61

In [123]:
_d, true, pred, mean = get_lr_local_interpretation(true_negative, lr_pred[true_negative][0], lr.coef_[-1])
fig = plot_local_interpretation(_d, f'Local interpretation of Linear Regression for a sample where ACTUAL={true} | PREDICTED={pred:.2f} | MEAN={mean:.2f}')
fig.show()

In [124]:
_d, true, pred, mean = get_lr_local_interpretation(false_negative, lr_pred[false_negative][0], lr.coef_[-1])
fig = plot_local_interpretation(_d, f'Local interpretation of Linear Regression for a sample where ACTUAL={true} | PREDICTED={pred:.2f} | MEAN={mean:.2f}')
fig.show()

In [125]:
_d, true, pred, mean = get_lr_local_interpretation(true_positive, lr_pred[true_positive][0], lr.coef_[-1])
fig = plot_local_interpretation(_d, f'Local interpretation of Linear Regression for a sample where ACTUAL={true} | PREDICTED={pred:.2f} | MEAN={mean:.2f}')
fig.show()

In [126]:
_d, true, pred, mean = get_lr_local_interpretation(false_positive, lr_pred[false_positive][0], lr.coef_[-1])
fig = plot_local_interpretation(_d, f'Local interpretation of Linear Regression for a sample where ACTUAL={true} | PREDICTED={pred:.2f} | MEAN={mean:.2f}')
fig.show()

## Lasso regression

## LARS

## Polynomial regression

## Linear regression with recursive feature elimination

## Polynomial regression with recursive feature elimination

## Comparison of all linear models

### Experiment configuration
- linear/logistic regression
- CrossNet (Wang et al. 2017)
  - number of units: 32
- XGBoost (Chen and Guestrin 2016)
  - number of boosting rounds: 1000
  - eta: 0.3
- EBM (Nori et al. 2019)
  - number of boosting rounds: 20000
  - inner bags: 8
  - outer bags: 8
  - learning rate: 0.01
- MLP
  - hidden layers: 3
  - hidden units: [32, 64, 32]
  - activation function: LeakyReLU
- NAM (Agarwal et al. 2020)
  - hidden layers: 3
  - hidden units: [32, 64, 32]
  - activation function: LeakyReLU
- NodeGAM (Chang, Caruana, and Goldenberg 2022), NodeGA2M
  - default hyperparameters provided by (Chang, Caruana, and Goldenberg 2022)
- HONAM
  - hidden layers: 3
  - hidden units: [32, 64, 32]
  - activation function: LeakyReLU
  
- epochs: 1000
- learning rate: 0.001
- batch size = (number of data samples)/100
- trained with 5 random seeds
- mean scores with standard deviations reported

## CrossNet
[deepctr-torch](https://deepctr-torch.readthedocs.io/en/latest/index.html)

In [82]:
from deepctr_torch.inputs import DenseFeat
from deepctr_torch.models import DCN

feature_columns = [DenseFeat(col, 1) for col in columns]

train_model_input = dict(zip(columns, x_train.T))
val_model_input = dict(zip(columns, x_val.T))
test_model_input = dict(zip(columns, x_test.T))
    
dcn = DCN(linear_feature_columns=feature_columns, dnn_feature_columns=feature_columns, dnn_hidden_units=[16, 32, 16], dnn_dropout=0.5, l2_reg_linear=0.0001, l2_reg_embedding=0.0001, l2_reg_cross=0.0001, l2_reg_dnn=0.2, task='regression', device=device)
dcn.compile("adam", "mse", metrics=['mse'])
dcn.fit(train_model_input, y_train, validation_data=(val_model_input, y_val), epochs=1000)

cpu
Train on 266 samples, validate on 88 samples, 2 steps per epoch


2it [00:00, 194.85it/s]


Epoch 1/1000
0s - loss:  11.3100 - mse:  9.6216 - val_mse:  3.8805


2it [00:00, 185.84it/s]


Epoch 2/1000
0s - loss:  10.7489 - mse:  7.0470 - val_mse:  3.7456


...

Epoch 998/1000
0s - loss:  0.3988 - mse:  0.4461 - val_mse:  0.5546


2it [00:00, 195.39it/s]


Epoch 999/1000
0s - loss:  0.3986 - mse:  0.3667 - val_mse:  0.5555


2it [00:00, 211.66it/s]

Epoch 1000/1000
0s - loss:  0.3985 - mse:  0.5244 - val_mse:  0.5586


In [83]:
print("TRAIN: ", evaluate(y_train, dcn.predict(train_model_input), task))
print("VAL: ", evaluate(y_val, dcn.predict(val_model_input), task))
print("TEST: ", evaluate(y_test, dcn.predict(test_model_input), task))

TRAIN:  (0.6152292435658568, 0.4351344981179265)
VAL:  (0.3674975434945934, 0.20927328729259853)
TEST:  (0.4160978094946234, 0.31908332397484973)


## Multi Layer Perceptron
[Tutorial](https://machinelearningmastery.com/building-multilayer-perceptron-models-in-pytorch/)

In [84]:
from sklearn.neural_network import MLPRegressor
# {'alpha': 0.0001,
#  'learning_rate': 'constant',
#  'learning_rate_init': 0.001,
#  'tol': 0.001
# }
mlp = MLPRegressor(hidden_layer_sizes=(32,64,32), batch_size=batch_size, learning_rate='adaptive', learning_rate_init=0.001, max_iter=1000, early_stopping=True, validation_fraction=0.2, tol=0.001, alpha=0.0001)
mlp.fit(np.concatenate((x_train, x_val)), np.concatenate((y_train, y_val)).ravel())

MLPRegressor(batch_size=3, early_stopping=True, hidden_layer_sizes=(32, 64, 32),
             learning_rate='adaptive', max_iter=1000, tol=0.001,
             validation_fraction=0.2)

In [88]:
print("TRAIN: ", evaluate(y_train, mlp.predict(x_train), task))
print("VAL: ", evaluate(y_val, mlp.predict(x_val), task))
print("TEST: ", evaluate(y_test, mlp.predict(x_test), task))

TRAIN:  (0.5386719402414666, 0.3552126761733151)
VAL:  (0.5337154868312967, 0.3099292550926127)
TEST:  (0.471832005365755, 0.3279240212277379)


## XGBoost
[dlmc](https://xgboost.readthedocs.io/en/stable/index.html)

In [99]:
from xgboost import XGBRegressor
xgb = XGBRegressor(n_estimators=1000, learning_rate=0.01, gamma=1.5, max_depth=3, objective='reg:squarederror', eval_metric='rmse')
xgb.fit(x_train, y_train)

print("TRAIN: ", evaluate(y_train, xgb.predict(x_train), task))
print("VAL: ", evaluate(y_val, xgb.predict(x_val), task))
print("TEST: ", evaluate(y_test, xgb.predict(x_test), task))
xgb_df = pd.DataFrame([(col, coefficient) for col, coefficient in zip(columns, xgb.feature_importances_)], columns=['variable', 'feature_importance'])
xgb_df['variable'] = xgb_df['variable'].apply(lambda x: labels[x])
xgb_df.set_index(['variable']).sort_values(by=['feature_importance'], ascending=False)

TRAIN:  (0.6922539508688266, 0.45811924094715883)
VAL:  (0.29913983702167024, 0.1490933496086072)
TEST:  (0.45476596407544734, 0.3183432898885715)


,feature_importance
variable,
body mass index,0.336355
lamotrigine,0.219906
glucose,0.083001
average blood pressure,0.071175
total cholesterol,0.069389
low-density lipoprotein,0.048221
high-density lipoprotein,0.048206
age,0.047109
total serum cholesterol,0.039210


In [100]:
import plotly.express as px
fig = px.bar(xgb_df, y='variable', x='feature_importance', orientation='h', range_x=(0, 0.5), text='variable', title='Global interpretation of Linear Regression', height=800, width=800, labels={'feature_importance': 'feature importance'})
fig.update_yaxes(visible=False, showticklabels=False)
fig.update_xaxes(showgrid=False)
fig.show()

## Explainable Boosting Machine
[interpret.ml](https://interpret.ml/docs/ebm.html#)

In [134]:
from sklearn.model_selection import GridSearchCV
from interpret.glassbox import ExplainableBoostingRegressor

tuned_parameters = {
    "learning_rate": [0.2, 0.5, 0.8]
}

grid_search = GridSearchCV(
    ExplainableBoostingRegressor(feature_names=columns, outer_bags=12, inner_bags=2, max_rounds=20_000, learning_rate=0.5, validation_size=0.2),
    tuned_parameters, scoring='r2', refit=False, n_jobs= -1, cv=3, verbose=1
)
grid_search.fit(np.concatenate((x_train, x_val)), np.concatenate((y_train, y_val)))
print(grid_search.best_params_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
{'learning_rate': 0.5}


In [141]:
from interpret.glassbox import ExplainableBoostingRegressor
ebm = ExplainableBoostingRegressor(feature_names=[labels[c] for c in columns], outer_bags=12, inner_bags=2, max_rounds=20_000, learning_rate=0.5, validation_size=0.2)
ebm.fit(np.concatenate((x_train, x_val)), np.concatenate((y_train, y_val)))
ebm_pred = ebm.predict(x_test)

print("TRAIN: ", evaluate(y_train, ebm.predict(x_train), task))
print("VAL: ", evaluate(y_val, ebm.predict(x_val), task))
print("TEST: ", evaluate(y_test, ebm.predict(x_test), task))

TRAIN:  (0.7037490654144816, 0.48035039855087835)
VAL:  (0.6820024330946233, 0.4128854972411623)
TEST:  (0.4511466167286653, 0.31261026985193063)


In [142]:
from interpret import show
show(ebm.explain_global(name='Global feature importance for Explainable Boosting Classifier'))

<!-- http://127.0.0.1:7127/140511462471712/ -->

In [143]:
show(ebm.explain_local(x_test[[true_negative, false_negative, true_positive, false_positive]], y_test[[true_negative, false_negative, true_positive, false_positive]], name='Local interpretation of Explainable Boosting Classifier'))

<!-- http://127.0.0.1:7127/140511462366272/ -->

## NAM
- [google-research/neural_additive_models](https://github.com/google-research/google-research/tree/master/neural_additive_models)
- [emeln/nam](https://github.com/lemeln/nam)

In [144]:
from nam.wrapper import NAMRegressor
nam = NAMRegressor(num_epochs=1000, num_learners=10, metric='mse', monitor_loss=True, n_jobs=-1, random_state=42, val_split=0.2)
nam.fit(np.concatenate((x_train, x_val)), np.concatenate((y_train, y_val)).ravel())

/Users/mtamkeen/opt/anaconda3/lib/python3.9/site-packages/nam/trainer/trainer.py:11: TqdmExperimentalWarning:

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)

2023-06-10 16:40:41.606020: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-10 16:40:41.606068: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-10 16:40:41.606095: I tensorflow/core/platform/cpu_feature_guard.cc:193] Th

In [147]:
print("TRAIN: ", evaluate(y_train, nam.predict(x_train), task))
print("VAL: ", evaluate(y_val, nam.predict(x_val), task))
print("TEST: ", evaluate(y_test, nam.predict(x_test), task))

TRAIN:  (0.5062966935238147, 0.31700748144064084)
VAL:  (0.503822290893633, 0.26918241883007976)
TEST:  (0.4438813717763743, 0.3014191447576321)


In [150]:
nam.plot(0).keys()

dict_keys(['x', 'y', 'conf_int'])

In [152]:
import plotly.graph_objects as go
fig = go.Figure()
for i, col in enumerate(columns):
    nam_plot = nam.plot(i)
    fig.add_trace(go.Scatter(x=nam_plot['x'], y=nam_plot['y'], mode='lines', name=f'{labels[col]}'))
fig.show()

## NODE GAM
[github](https://github.com/zzzace2000/nodegam)

In [155]:
from nodegam.sklearn import NodeGAMRegressor

nodegam = NodeGAMRegressor(len(columns), validation_size=0.2, ga2m=0, device=device)
history = nodegam.fit(pd.DataFrame(np.concatenate((x_train, x_val)), columns=columns), np.concatenate((y_train, y_val)).ravel())

/Users/mtamkeen/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2627: UserWarning:

n_quantiles (2000) is greater than the total number of samples (354). n_quantiles is set to n_samples.

/Users/mtamkeen/opt/anaconda3/lib/python3.9/site-packages/nodegam/odst.py:145: UserWarning:

Data-aware initialization is performed on less than 1000 data points. This may cause instability. To avoid potential problems, run this model on a data batch with at least 1000 data samples. You can do so manually before training. Use with torch.no_grad() for memory efficiency.



Normalize y. mean = -0.04733101025388134, std = 0.9887955253006494


/Users/mtamkeen/opt/anaconda3/lib/python3.9/site-packages/qhoptim/pyt/qhadam.py:133: UserWarning:

This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/python_arg_parser.cpp:1485.)



Steps	Train Err	Val Metric (mse)
100	0.4838	1.8602
200	0.5381	0.914
300	0.7511	0.7942
400	0.4268	0.6688
500	0.3643	0.7268
600	0.3777	0.9042
700	0.3134	0.8522
800	0.2865	1.1291
900	0.3618	1.2027
1000	0.4109	1.148
1100	0.2811	1.1554
1200	0.2897	1.2935
1300	0.2813	1.1711
1400	0.2641	0.9265
1500	0.2908	0.9137
1600	0.2955	0.774
1700	0.3003	0.8172
1800	0.3281	0.8478
1900	0.3555	0.6988
2000	0.3397	0.6706
2100	0.3178	0.6672
2200	0.2997	0.6811
2300	0.2693	0.6924
2400	0.2957	0.7162
2500	0.2677	0.7441
2600	0.2702	0.752
2700	0.2505	0.7722
LR: 1.00e-02 -> 2.00e-03
2800	0.2204	0.7847
2900	0.262	0.782
3000	0.2369	0.7836
3100	0.2339	0.8012
3200	0.2358	0.806
3300	0.2219	0.8103
LR: 2.00e-03 -> 4.00e-04
3400	0.2404	0.8097
3500	0.2357	0.8133
3600	0.2279	0.8124
3700	0.2131	0.8057
3800	0.2511	0.8025
3900	0.2472	0.8037
LR: 4.00e-04 -> 8.00e-05
4000	0.2183	0.8025
4100	0.2222	0.8011
BREAK. There is no improvment for 2000 steps
Total training time: 249.5 seconds
Best step:  2100
Best Val Metric:  0.667204746009

{'train_losses': [1.93012273311615,
  1.7038837671279907,
  1.3074331283569336,
  0.9966943860054016,
  2.177330255508423,
  1.348001480102539,
  1.261730670928955,
  1.4661613702774048,
  1.081825613975525,
  1.1223814487457275,
  0.9304205775260925,
  1.0195868015289307,
  1.0716854333877563,
  1.4771093130111694,
  0.8960811495780945,
  0.9863197207450867,
  0.992305338382721,
  0.9253068566322327,
  0.9084256887435913,
  1.2006728649139404,
  0.7959650754928589,
  0.6985369324684143,
  0.6666201949119568,
  0.6687647104263306,
  0.864900529384613,
  0.7478764057159424,
  0.7247359156608582,
  1.6421605348587036,
  0.6437190175056458,
  0.6979851126670837,
  0.8285125494003296,
  0.7165687084197998,
  0.9472308158874512,
  0.6873120665550232,
  1.5835418701171875,
  1.0039875507354736,
  0.6134284138679504,
  1.0128355026245117,
  0.701561450958252,
  0.6029914617538452,
  0.7640587091445923,
  0.664871096611023,
  0.6429435610771179,
  0.9472283124923706,
  0.5915856957435608,
  0.

In [156]:
print("TRAIN: ", evaluate(y_train, nodegam.predict(pd.DataFrame(x_train, columns=columns)), task))
print("VAL: ", evaluate(y_val, nodegam.predict(pd.DataFrame(x_val, columns=columns)), task))
print("TEST: ", evaluate(y_test, nodegam.predict(pd.DataFrame(x_test, columns=columns)), task))

TRAIN:  (0.6403909393346567, 0.4563624242044435)
VAL:  (0.6208576526532297, 0.37682894904947406)
TEST:  (0.348950503735842, 0.27253734126453344)


## NODE GA2M

In [159]:
from nodegam.sklearn import NodeGAMRegressor

nodega2m = NodeGAMRegressor(len(columns), validation_size=0.2, ga2m=1, device=device)
history = nodega2m.fit(pd.DataFrame(np.concatenate((x_raw_train, x_raw_val)), columns=columns), np.concatenate((y_raw_train, y_raw_val)).ravel())

/Users/mtamkeen/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2627: UserWarning:

n_quantiles (2000) is greater than the total number of samples (354). n_quantiles is set to n_samples.

/Users/mtamkeen/opt/anaconda3/lib/python3.9/site-packages/nodegam/odst.py:145: UserWarning:

Data-aware initialization is performed on less than 1000 data points. This may cause instability. To avoid potential problems, run this model on a data batch with at least 1000 data samples. You can do so manually before training. Use with torch.no_grad() for memory efficiency.



Normalize y. mean = 153.82203389830508, std = 77.8576454873573
Steps	Train Err	Val Metric (mse)
100	0.4354	7681.6211
200	0.4339	5030.3114
300	0.4274	4932.002
400	0.4019	4398.7727
500	0.4089	4280.7942
600	0.3255	4164.7212
700	0.3009	4710.9067
800	0.2896	4954.5352
900	0.2785	4950.801
1000	0.2968	4912.3896
1100	0.3226	4832.7043
1200	0.244	4968.3272
1300	0.2581	5289.886
1400	0.2849	5761.5332
1500	0.1935	5951.698
1600	0.2327	6087.6844
1700	0.2814	6044.3562
1800	0.2317	6174.5583
1900	0.1758	6064.147
2000	0.1943	6345.709
2100	0.1967	6435.946
2200	0.219	6646.4371
2300	0.1789	6821.8924
2400	0.1653	6868.0233
2500	0.1357	6869.3783
2600	0.1528	6908.2974
LR: 1.00e-02 -> 2.00e-03
2700	0.1236	6936.5621
2800	0.1385	6959.5575
2900	0.13	6867.9626
3000	0.0895	6835.8218
3100	0.1003	6888.397
3200	0.1866	6897.7485
LR: 2.00e-03 -> 4.00e-04
3300	0.087	6947.4109
3400	0.0901	6962.1336
3500	0.1107	6986.5092
3600	0.115	6964.386
3700	0.1069	6955.9364
3800	0.1037	6953.7642
LR: 4.00e-04 -> 8.00e-05
3900	0.1016	6958.

In [162]:
print("TRAIN: ", evaluate(y_raw_train, nodega2m.predict(pd.DataFrame(x_raw_train, columns=columns)), task))
print("VAL: ", evaluate(y_raw_val, nodega2m.predict(pd.DataFrame(x_raw_val, columns=columns)), task))
print("TEST: ", evaluate(y_raw_test, nodega2m.predict(pd.DataFrame(x_raw_test, columns=columns)), task))

TRAIN:  (0.4933746229813868, 0.3438632481542112)
VAL:  (0.433808324107276, 0.22651101313571054)
TEST:  (0.3539399464367615, 0.2716975927514237)


## HONAM

In [163]:
from model import HONAM
honam = HONAM(
        num_features=x_train.shape[1],
        out_size=1,
        task=task,
        order=2,
        verbose=False,
        lr=0.0001,
        feature_net_dropout=0.2,
        batch_size=200,
        epochs=1000
    ).to(device)
honam.fit(x_train, y_train, x_val, y_val)

In [164]:
print('TRAIN: ', evaluate(y_train, honam.predict(x_train), task))
print('VAL: ', evaluate(y_val, honam.predict(x_val), task))
honam_pred = honam.predict(x_test)
print('TEST: ', evaluate(y_test, honam_pred, task))

TRAIN:  (0.6734530749446624, 0.4712444180285139)
VAL:  (0.42582414935942914, 0.228770256237391)
TEST:  (0.4693771582917068, 0.30607441810659686)


In [ ]:
x_test[0].reshape(-1,1).dtype

In [ ]:
honam.forward(torch.from_numpy(x_test[0].reshape(-1,1)))

In [ ]:
import plotly.figure_factory as ff
fig = ff.create_distplot([y_test.ravel(), honam.predict(x_test).ravel()], ['actual', 'predicted'], show_hist=False)
fig.show()

In [ ]:
abs(-1)

In [ ]:
y_i = 100
r2 = []
r_abs = []
x = []
for y_hat in range(-50, 251):
    r2.append(1 - ((y_i - y_hat)**2 / y_i**2))
    r_abs.append(1 - (abs(y_i - y_hat) / y_i))
    x.append(y_hat)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=r2, mode='lines', name='R-squared'))
fig.add_trace(go.Scatter(x=x, y=r_abs, mode='lines', name='R-absolute'))
fig.add_vline(x=100, line_dash="dash", annotation_text="actual value", annotation_position="bottom right")
fig.update_layout(title='Change in scores with different prediction errors', xaxis_title='predicted value', yaxis_title='score', height=800)


fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=r_abs[:151], y=r2[:151], mode='lines', name='R-squared'))
# fig.update_layout(title='Change in scores with different prediction errors', xaxis_title='predicted value', yaxis_title='score')
fig.show()